In [ ]:
import os
import math
import numpy as np
import pandas as pd
import QuantLib as ql
from scipy.optimize import minimize
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

## basic funtions for interest rates, date form processing
nasdaq_holidays = [
    '2018-01-01', '2018-01-15', '2018-02-19', '2018-03-30', '2018-05-28',
    '2018-07-04', '2018-09-03', '2018-11-22', '2018-12-25',
    '2019-01-01', '2019-01-21', '2019-02-18', '2019-04-19', '2019-05-27',
    '2019-07-04', '2019-09-02', '2019-11-28', '2019-12-25',
    '2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25',
    '2020-07-03', '2020-09-07', '2020-11-26', '2020-12-25',
    '2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31',
    '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24',
    '2022-01-01', '2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30',
    '2022-07-04', '2022-09-05', '2022-11-24', '2022-12-26',
    '2023-01-01', '2023-01-16', '2023-02-20', '2023-04-07', '2023-05-29',
    '2023-07-04', '2023-09-04', '2023-11-23', '2023-12-25'
]
nasdaq_holidays = np.array(nasdaq_holidays, dtype='datetime64[D]')


def IR_effect(init_date, start_date, end_date):
    if start_date == end_date:
        return 0.0
    else:
        tau = np.busday_count(start_date, end_date, holidays=nasdaq_holidays) / 252
        if tau == 0.0:
            return 0.0
        else:
            beta0 = IRParams.loc[IRParams.Date <= init_date, 'BETA0'].values[-1]
            beta1 = IRParams.loc[IRParams.Date <= init_date, 'BETA1'].values[-1]
            beta2 = IRParams.loc[IRParams.Date <= init_date, 'BETA2'].values[-1]
            tau1 = IRParams.loc[IRParams.Date <= init_date, 'TAU1'].values[-1]

            r = beta0 + beta1*(1-math.exp(-tau/tau1))/(tau/tau1) + beta2*((1-math.exp(-tau/tau1))/(tau/tau1)-math.exp(-tau/tau1))
            return r/100
def date_asQuantLib(t):
    result = pd.to_datetime(t)
    return ql.Date(result.day, result.month, result.year)

### Black-Scholes model calibration with Quantlib and Tikhonov regularization

In [ ]:
# Define a helper to price options using the Heston model
def model_price(strike, expiry, model):
    payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
    exercise = ql.EuropeanExercise(expiry)
    option = ql.VanillaOption(payoff, exercise)
    engine = ql.AnalyticEuropeanEngine(model)
    if engine is None:
        print("Pricing engine is not initialized properly.")
    option.setPricingEngine(engine)
    return option.NPV()


# Define the objective function (least squares + Tikhonov regularization)
def objective_function(params, market_data, alpha, prior_params):
    volatility = params[0]
    strikes = market_data['strike_prices']
    market_prices = market_data['market_prices']
    expiries = market_data['expiry']
    rf = market_data['risk_free_curve']
    volatility_curve = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(market_data['valuationDate'], calendar, volatility, day_count))
    bsm_model = ql.BlackScholesMertonProcess(
        market_data['spot_price'], market_data['dividend_yield'], rf, volatility_curve)

    # Calculate the sum of squared errors between model and market prices
    errors = np.sum([(model_price(strike, expiry, bsm_model) - market_price)**2
                     for strike, market_price, expiry in zip(strikes, market_prices, expiries)])

    # Tikhonov regularization (penalty for deviating from prior guess)
    regularization = alpha * np.sum((params - prior_params)**2)

    return errors + regularization


# Calibrate the parameters on each t0
def calibrateBS(df, t0, S0, r0, init_params):
    '''
    Main funtion
    df: dataframe contains the call option prices at t0
    t0: date of calibration %Y%m%d
    S0: original stock price
    r0: interest rate at t0 to T2
    init_params: array([sigma])
    '''
    valuation_date = t0
    ql.Settings.instance().evaluationDate = valuation_date
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(S0))
    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, day_count))
    risk_free_curve = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, r0, day_count))

    market_data = {
        'strike_prices': df['K'],
        'market_prices': (df['ask']+df['bid'])/2,  # Observed market option prices
        'expiry': df['t'],
        'spot_price': spot_handle,
        'dividend_yield': dividend_yield,
        'risk_free_curve': risk_free_curve,
        'valuationDate': valuation_date
    }

    initial_guess = init_params
    prior_params = init_params  # Prior guess for Tikhonov
    alpha = 0.01  # Regularization parameter

    result = minimize(
        objective_function,
        x0=initial_guess,
        args=(market_data, alpha, prior_params),
        method='L-BFGS-B',  # Optimization method, can use other methods
        bounds=[(0.00001, 1)])
    return result.x

### Daily Delta-Hedging under Black-Scholes model

In [ ]:
def calc_Payoff(S1, S2):
    return np.maximum(S2 - S1, 0)

def vanillaOptionPricer(S0, T_t, K, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T_t) / (sigma * np.sqrt(T_t))
    d2 = d1 - sigma * np.sqrt(T_t)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T_t) * norm.cdf(d2)

def forwardStartOptionPricer(T1, T2, params, S0, r, k=1):
    return np.exp(-k*T1)*vanillaOptionPricer(S0,T2-T1,k*S0,r,params[0])

def calc_GeekingHedge(T1, T2, params, S0, r, alpha, K=None):
    '''
    Delta hedging calculated as (P(S*(1+alpha))-P(S))/(alpha*S)
    '''
    if K != None:
      delta = vanillaOptionPricer(S0 * (1 + alpha), T2-T1, K, r, params[0]) - vanillaOptionPricer(S0, T2-T1, K, r, params[0])
    else:
      delta = forwardStartOptionPricer(T1, T2, params, S0 * (1 + alpha), r) - forwardStartOptionPricer(T1, T2, params, S0, r)
    return delta / (alpha * S0)

def BlackScholes_main(item):
    '''
    Main function for delta-hedging
    item: pair(ticker_T1_T2),data(contains daily vanilla call option prices)
    '''
    pair, data = item
    print(pair)
    ticker = pair.split('_')[0].lower()
    stock_df = pd.read_csv('data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker), index_col=0)
    stock_df = stock_df.set_index('date')

    t0List = data['t0'].unique()
    t0List.sort()
    params_dict = {}
    gap_list = []
    for i in range(len(t0List)-1,-1,-1):
        t0 = t0List[i]
        df = data[data['t0'] == t0]

        ##change to proper data form 
        df1 = df[['t0', 'T1', 'K1', 'C1_ask', 'C1_bid']]
        df2 = df[['t0', 'T2', 'K2', 'C2_ask', 'C2_bid']]
        df1.columns = ['t0', 't', 'K', 'ask', 'bid']
        t1 = df1['t'].unique()[0]
        df1['t'] = date_asQuantLib(t1)
        df2.columns = ['t0', 't', 'K', 'ask', 'bid']
        t2 = df2['t'].unique()[0]
        df2['t'] = date_asQuantLib(t2)
        df = pd.concat([df1, df2]).dropna()

        ## prepare stock price, BS model sigma, intereset rate
        S0 = stock_df.loc[t0]['adjusted_price']
        r = IR_effect(t0, t0, t2)
        params = calibrateBS(df,date_asQuantLib(t0),S0,r,init_params)
        params_dict[t0] = params
        payoff_Actual = calc_Payoff(stock_df.loc[t1, 'adjusted_price'], stock_df.loc[t2, 'adjusted_price'])

        ##Hedging
        T1 = day_count.yearFraction(date_asQuantLib(t0), date_asQuantLib(t1))
        T2 = day_count.yearFraction(date_asQuantLib(t0), date_asQuantLib(t2))
        S_t1 = stock_df.loc[t1:].iloc[0, 0]
        price = forwardStartOptionPricer(T1, T2, params, S0, r)
        t_i = pd.to_datetime(t0)
        # ti_str = t_i.strftime('%Y-%m-%d')
        while t_i < pd.to_datetime(t2):
            ti_str = t_i.strftime('%Y-%m-%d')
            S_i = stock_df.loc[ti_str:].iloc[0, 0]
            r = IR_effect(ti_str, ti_str, t2)
            params_hedge = params_dict[max([date for date in t0List if date <= ti_str])]
            if T1 <= 0:
                delta_i = calc_GeekingHedge(0, T2, params_hedge, S_i, r, alpha=0.0001 * S0, K=S_t1)
            else:
                delta_i = calc_GeekingHedge(max(T1, 0), T2, params_hedge, S_i, r, alpha=0.0001 * S0)
            t_i += pd.Timedelta(days=1)
            T1 = day_count.yearFraction(date_asQuantLib(t_i), date_asQuantLib(t1))
            T2 = day_count.yearFraction(date_asQuantLib(t_i), date_asQuantLib(t2))
            t_i = min(t_i, pd.to_datetime(t2))
            price += delta_i * (stock_df.loc[:t_i.strftime('%Y-%m-%d')].iloc[-1, 0] - S_i)

        gap_list.append((price - payoff_Actual) / S0)
    timeToMaturity = np.busday_count(pd.to_datetime(t0List[::-1]).values.astype('datetime64[D]'),
                                     pd.to_datetime([t1]).values.astype('datetime64[D]'),
                                     holidays=nasdaq_holidays)
    gap_list = pd.DataFrame(gap_list, index=timeToMaturity, columns=[pair]).T
    gap_list.to_csv('result_BS/{}.csv'.format(pair))
    return gap_list
    

In [ ]:
if __name__ == "__main__":
    sigma = 0.1  # Volatility
    init_params = np.array([sigma])

    IRParams = pd.read_csv('interest_rates_parameters.csv', parse_dates=['Date'], dayfirst=True)
    IRParams=IRParams.sort_values(by='Date')
    IRParams = IRParams.fillna(method='ffill')

    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    day_count = ql.Actual365Fixed()

    data_path = f'data/options_call_askbid/'
    paths = os.listdir(data_path)
    paths = [file for file in paths if file[-4:] == '.csv']
    gap_df = pd.DataFrame()

    data_dict = {}
    for path in paths:
        data = pd.read_csv(data_path+path, index_col=0)
        pair = path.split('.')[0].split('/')[-1]
        if pair not in gap_df.index:
            data_dict[pair] = data

    for item in data_dict.items():
        BlackScholes_main(item)
        